In [3]:
import json

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

oai_client = OpenAI(api_key =os.getenv("LANDPRO_OAI_KEY"))

load_dotenv('/Users/paigegiese/SYG/landproDATA_code/misc-work/.env')

username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/landprodata")
query = "SELECT * FROM entities"
df = pd.read_sql(query, con=engine)
df.head()

,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,Is Name Confirmed by LPD,IDSOS Filing Number,IDSOS Control ID,Type,...,Zip Code,IDSOS Filing Date,Industry Types,Company Size,Status,URL,Source Comments,landproDATA Comments,Entered By,Entered Time
0,1,Firm,Surveyor,Survey,2 Bar K Land Surveying,2 Bar K Land Surveying,0,,,,...,,1969-12-31,,,1,,,no IDSOS filing number,Barbara Broussard,2014-10-07
1,2,Firm,Surveyor,Survey,4 Rivers Surveying Inc.,4 RIVERS SURVEYING INC.,1,C184933,558923,Corp,...,83617,2009-10-27,,None,99,https://www.4rivers-surveying.com/,"GOODSTANDING, ANREPT SENT 11 Aug 2014",full-service land surveying company specializ...,Barbara Broussard,2014-10-07
2,3,Firm,Developer,RE - Development,A Team Land Consultants LLC,A TEAM LAND CONSULTANTS LLC,1,W101200,312892,LLC,...,83709,2011-03-07,,None,1,None,"EXISTING, ANREPT SENT 08 Jan 2014",,Barbara Broussard,2014-10-07
3,4,Firm,Surveyor,Survey,"Accurate Surveying & Mapping, P.C.","ACCURATE SURVEYING & MAPPING, P.C.",1,C163074,494817,Corp,...,83703,2005-10-24,,None,1,None,"GOODSTANDING, ANREPT SENT 11 Aug 2014",None,Barbara Broussard,2014-10-07
4,5,Agency,Public Agency,,Ada County Highway District,,0,,,Public Agency,...,,1969-12-31,,,2,,,,Barbara Broussard,2014-10-07


In [4]:
with open("entities_reclassed_v1.jsonl", "r") as f:
    data = [json.loads(line) for line in f if line.strip()]

In [9]:
reclassified = pd.DataFrame()
for d in data:
    reclassified = pd.concat([reclassified, pd.DataFrame(d)]).reset_index(drop=True)

In [13]:
df[['ID',
 'Entity Group ID',
 'Entity Subgroup ID',
 'Entity Category',
 'Name',
 'IDSOS Name']]

,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name
0,1,Firm,Surveyor,Survey,2 Bar K Land Surveying,2 Bar K Land Surveying
1,2,Firm,Surveyor,Survey,4 Rivers Surveying Inc.,4 RIVERS SURVEYING INC.
2,3,Firm,Developer,RE - Development,A Team Land Consultants LLC,A TEAM LAND CONSULTANTS LLC
3,4,Firm,Surveyor,Survey,"Accurate Surveying & Mapping, P.C.","ACCURATE SURVEYING & MAPPING, P.C."
4,5,Agency,Public Agency,,Ada County Highway District,
...,...,...,...,...,...,...
13390,16309,None,,,Self,None
13391,16311,None,,,landproDATA Inc.,None
13392,16312,None,,,AMP,None
13393,16313,None,,,"landproDATA, Inc",None


In [22]:
reclassified2 = reclassified.loc[~reclassified['original_id'].isin(reclassified['original_id'].value_counts().head(3).index.tolist())].reset_index(drop=True)
reclassified2['original_id'] = reclassified2['original_id'].astype(int)

In [25]:
reclassified3 = reclassified2.merge(df[['ID',
 'Entity Group ID',
 'Entity Subgroup ID',
 'Entity Category',
 'Name',
 'IDSOS Name']], how='inner', left_on='original_id', right_on='ID')

In [27]:
reclassified3['entity_group_id'].value_counts()

entity_group_id
Private    7804
Public      184
Name: count, dtype: int64

In [28]:
reclassified3['entity_category'].value_counts()

entity_category
Contractor                4514
Real Estate                752
Builder                    693
Other                      616
Engineering                406
Surveying                  291
Agency                     196
Architect                  134
Appraisal                  134
Inspection                  50
Supply                      39
Design                      28
Homeowners Association      24
Remodeling                  18
Restoration                 17
Title                       16
Landscape                   15
Environmental               12
Electrical                  11
Sign                         6
Communications               6
Trucking                     3
Developer                    2
Stone                        1
Testing                      1
Excavation                   1
Security                     1
Planning                     1
Name: count, dtype: int64

In [30]:
reclassified3['entity_subgroup_id'].value_counts()[:20]

entity_subgroup_id
General         2291
NULL            1507
Residential      907
Electrical       429
Plumbing         277
                 244
HVAC             227
Landscape        171
Roofing          133
Excavation       122
Water            102
Civil            101
Commercial        81
Remodeling        71
Mechanical        63
Full Service      60
Irrigation        57
Concrete          56
Fire              48
Home              47
Name: count, dtype: int64

In [51]:
reclassified3 = reclassified3.replace('',None).replace('NULL',None)

In [58]:
reclassified3['entity_category'].value_counts()

entity_category
Contractor                4514
Real Estate                752
Builder                    693
Other                      616
Engineering                406
Surveying                  291
Agency                     196
Architect                  134
Appraisal                  134
Inspection                  50
Supply                      39
Design                      28
Homeowners Association      24
Remodeling                  18
Restoration                 17
Title                       16
Landscape                   15
Environmental               12
Electrical                  11
Sign                         6
Communications               6
Trucking                     3
Developer                    2
Stone                        1
Testing                      1
Excavation                   1
Security                     1
Planning                     1
Name: count, dtype: int64

In [59]:
rules = [{'Entity Category': 'Surveying', 'Entity Subgroup ID': "Subgroup shall be left NULL",'commentary': "will be separate from engineering unless included within an Engineering. See Engineering Category rule."},
{'Entity Category': 'Contractor', 'Entity Subgroup ID': '<insert specialty here>. If no specialty, denote "General"','commentary': "Shall include records that indicate construction as a general contractor or sub contractor. \
 In the case of a specialty contracting, like 'HVAC', specify the specialty in the 'Entity Subgroup ID' key.\
 If 'Construction' is in the business name, Entity Category is 'Contractor', regardless of 'Builder' being present in either the Entity Subgroup ID or Entity Category of the INPUT DATA."},
{'Entity Category': 'Builder', 'Entity Subgroup ID': "'Residential', 'Commercial', or NULL if both. If 'Homes' is in the compny name, it is safe to assume 'Residential' as long as there is 0 indication of commercial activity.",'commentary': "Home builders are denoted with 'Residential' subgroup. If there is no indication of being a Builder in the input data, defer the classification to Contractor."},
{'Entity Category': 'Engineering', 'Entity Subgroup ID': "Insert specialty(ies) if known. 'Full service', if known.",'commentary': "Shall include all different engineering professions and specialities. The subgroup will be 'Full Service' if specified in input data. Denote the specialty (e.g. 'Civil') in Subgroup ID. If multiple, separate the specialities using commas."},
{'Entity Category': 'Real Estate', 'Entity Subgroup ID': "Subgroup will be NULL unless otherwise specified as 'Residential','Commercial','Land', or a comma separated combination of those values. 'Land' can be used as a Subgroup to identify Land Brokers.",'commentary': "Expand 'RE' in the input data to 'Real Estate'"},
{'Entity Category': 'Agency', 'Entity Subgroup ID': "Attempt the following valid values: 'Water', 'Road', Assessor', 'Municipality', 'Land', 'Health', 'Urban Renewal', 'Fire', 'Ancillary'. ONLY use Ancillary if no other category makes sense in context.",'commentary': "NOTE: all Agencies should be marked 'Public' in the Entity Group ID key."},
{'Entity Category': 'Title', 'Entity Subgroup ID': "NULL",'commentary': "Used to identify Private title companies."},
{'Entity Category': 'Architect', 'Entity Subgroup ID': "NULL unless 'Landscape', then place 'Landscape' as the Subgroup.",'commentary': ""},
{'Entity Category': 'Inspection', 'Entity Subgroup ID': "Attempt the following value values: 'Home','Electrical','Fire','Health','Other'. Use 'Other' if none of the valid values make sense.",'commentary': ""},
{'Entity Category': 'Appraisal','Entity Subgroup ID': "NULL", 'commentary':"you will find 'Appraiser' often in the input data"},
{'Entity Category': 'Homeowners Association','Entity Subgroup ID': "NULL", 'commentary':"Note that values 'HOA' are equivalent. Translate to 'Homeowners Association'"},
{'Entity Category': 'Other','Entity Subgroup ID': "NULL", 'commentary':"Use only in the event that you are not confident in the reclassification."}]
cats =[]
for r in rules:
    cats.append(r['Entity Category'])

In [62]:
cats= cats[:-1]

In [63]:
for i, j in reclassified3.iterrows():
    if j['entity_category'] not in cats:
        reclassified3.at[i,'entity_category'] = 'Other'

In [71]:
reclassified3['entity_subgroup_id'].value_counts(normalize=True).cumsum()[:20]

entity_subgroup_id
General         0.367324
Residential     0.512747
Electrical      0.581530
Plumbing        0.625942
HVAC            0.662338
Landscape       0.689755
Roofing         0.711079
Excavation      0.730640
Water           0.746994
Civil           0.763187
Commercial      0.776174
Remodeling      0.787558
Mechanical      0.797659
Full Service    0.807279
Irrigation      0.816418
Concrete        0.825397
Fire            0.833093
Home            0.840629
Municipality    0.848004
Land            0.854738
Name: proportion, dtype: float64

Add backs from original

In [65]:
not_in = df.loc[~df['ID'].isin(reclassified3['ID'].tolist())][['ID',
 'Entity Group ID',
 'Entity Subgroup ID',
 'Entity Category',
 'Name',
 'IDSOS Name']].replace('', None)

for c in ['original_id', 'entity_group_id', 'entity_category',
       'entity_subgroup_id', 'confidence', 'candidates']:
       not_in[c] = None

full_table = pd.concat([reclassified3, not_in[['original_id', 'entity_group_id', 'entity_category',
       'entity_subgroup_id', 'confidence', 'candidates', 'ID',
       'Entity Group ID', 'Entity Subgroup ID', 'Entity Category', 'Name',
       'IDSOS Name']]])

/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_28262/1817622094.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_table = pd.concat([reclassified3, not_in[['original_id', 'entity_group_id', 'entity_category',


Status Determination

In [79]:
# 0 = declined to process for quality issues (i.e. should be removed) .. includes Landpro records and inactives
# 1 = declined to process for 'developer' review = audit
# 2 = declined to process because of too little initial information (null,null) = audit
# 3 = declined to process because they were marked 'consultant' = audit 
# 4 = failed to process in first run -> needs re-run
# 5 = builder -> not builder = audit
# 6 = entity_category marked 'Other' = audit
# 7 = processed and leaving in... = no audit

full_table['status'] = np.nan

# 1
devs=(full_table['Entity Category'].fillna('').str.contains('eveloper')
                    | full_table['Entity Subgroup ID'].fillna('').str.contains('eveloper'))
# 2
nullnull = (full_table['Entity Category'].isna() & full_table['Entity Subgroup ID'].isna())

# 0
inactive = (full_table['Name'].fillna('').str.contains('INACTIVE') |full_table['Name'].fillna('').str.contains('INATIVE'))

# 3
consultants=(full_table['Entity Category'].fillna('').str.contains('onsultant')
                    | full_table['Entity Subgroup ID'].fillna('').str.contains('onsultant')
                    | full_table['Name'].fillna('').str.contains('onsultant'))
# 0
lpd = full_table['Name'].fillna('').str.lower().str.contains('landpro')

# 5
not_builders = (((full_table['Entity Category']=='Builder') | (full_table['Entity Subgroup ID']=='Builder')) & ((full_table['entity_category']!= 'Builder') & full_table['entity_category'].notnull()))

full_table.loc[devs, 'status'] = 1


In [80]:
full_table.loc[nullnull, 'status'] = 2

In [81]:
full_table.loc[inactive, 'status'] = 0

In [82]:
full_table.loc[consultants, 'status'] = 3

In [83]:
full_table.loc[lpd, 'status'] = 0
full_table.loc[not_builders, 'status'] = 5

In [84]:
full_table['status'].value_counts()

status
2.0    2161
0.0    1899
1.0    1136
5.0     376
3.0      40
Name: count, dtype: int64

In [ ]:
full_table.loc[(full_table['status'].isna() & full_table['entity_category'].isna()), 'status'] = 4

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
0,1,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",1,Firm,Surveyor,Survey,2 Bar K Land Surveying,2 Bar K Land Surveying,NaN
1,2,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",2,Firm,Surveyor,Survey,4 Rivers Surveying Inc.,4 RIVERS SURVEYING INC.,NaN
2,4,Private,Surveying,None,98.0,"[{'entity_group_id': 'Private', 'entity_catego...",4,Firm,Surveyor,Survey,"Accurate Surveying & Mapping, P.C.","ACCURATE SURVEYING & MAPPING, P.C.",NaN
3,5,Public,Agency,Road,99.0,"[{'entity_group_id': 'Public', 'entity_categor...",5,Agency,Public Agency,None,Ada County Highway District,None,NaN
4,6,Public,Agency,Land,95.0,"[{'entity_group_id': 'Public', 'entity_categor...",6,Agency,Public Agency,None,Ada County Surveyor's Office,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7983,15757,Private,Engineering,Full Service,98.0,[],15757,Firm,Engineering - Full Service,Engineering - Full Service,Project Delivery Group (PDG),None,NaN
7984,15758,Private,Engineering,None,95.0,[],15758,Firm,Engineering,Engineering,Raymond J Briggs and Associates,None,NaN
7985,15794,Private,Engineering,"Civil, Planning, Surveying",90.0,"[{'entity_group_id': 'Private', 'entity_catego...",15794,Firm,"Engineer, Planner, Surveyor",None,Wilding Engineering,Wilding Engineering Inc.,NaN
7986,15801,Private,Surveying,None,95.0,"[{'entity_group_id': 'Private', 'entity_catego...",15801,Firm,"Surveying, Mapping",Survey,Enclosure Surveying,None,NaN


In [97]:
full_table.loc[(full_table['entity_category']=='Other')
| (full_table['confidence']<=70) & (full_table['entity_category'].fillna('')!= 'Other'),'status'] = 6

In [104]:
full_table.loc[full_table['status'].isna(),'status'] = 7

In [127]:
print(full_table['status'].value_counts())
print(full_table['status'].isin([5]).sum())

status
7.0    6788
2.0    2161
0.0    1899
1.0    1136
6.0     842
5.0     358
4.0     171
3.0      40
Name: count, dtype: int64
358


In [144]:
print(full_table.loc[(full_table['status']==1) & (full_table['Entity Category'].str.contains('eveloper'))]['Entity Category'].value_counts())
print(' ')
print(full_table.loc[(full_table['status']==1) & (full_table['Entity Subgroup ID'].str.contains('eveloper'))]['Entity Subgroup ID'].value_counts())

full_table.loc[(full_table['status']==1) & (full_table['Entity Category'].str.contains('eveloper'))]['Entity Category'].value_counts().index.tolist() +\
    full_table.loc[(full_table['status']==1) & (full_table['Entity Subgroup ID'].str.contains('eveloper'))]['Entity Subgroup ID'].value_counts().index.tolist()

Entity Category
Developer                    277
Builder/Developer             31
RE - Commercial/Developer      6
Builder, Developer             6
Builder Developer              2
Developer,Builder              2
Name: count, dtype: int64
 
Entity Subgroup ID
Developer                     945
Builder, Developer             72
Developer, Builder             42
Developer/Investments           8
Owner, Developer                6
Developer, Engineer             5
Developer, Owner                2
Builder, Developer, CommRE      1
Name: count, dtype: int64


['Developer',
 'Builder/Developer',
 'RE - Commercial/Developer',
 'Builder, Developer',
 'Builder Developer',
 'Developer,Builder',
 'Developer',
 'Builder, Developer',
 'Developer, Builder',
 'Developer/Investments',
 'Owner, Developer',
 'Developer, Engineer',
 'Developer, Owner',
 'Builder, Developer, CommRE']

In [130]:
full_table[full_table['status']==5]['Name'].apply(lambda n: 'onstruction' in n).mean()

np.float64(0.7039106145251397)

Need reprocessed because they errored in initial run OR had null, null in initial run, and now we're trying just based on name... 

In [156]:
full_table.loc[full_table['status'].isin([2])].sample(10)#['status'].value_counts()

,original_id,entity_group_id,entity_category,entity_subgroup_id,confidence,candidates,ID,Entity Group ID,Entity Subgroup ID,Entity Category,Name,IDSOS Name,status
12517,None,None,None,None,NaN,None,15287,None,None,None,Magnify Land Consulting LLC,Magnify Land Consulting LLC,2.0
12673,None,None,None,None,NaN,None,15479,None,None,None,Slice Ventures,None,2.0
12425,None,None,None,None,NaN,None,15194,Firm,None,None,BHEG Milestone Ranch LLC,BHEG Milestone Ranch LLC,2.0
12589,None,None,None,None,NaN,None,15374,None,None,None,LIVE MARKET ANALYTICS,None,2.0
13212,None,None,None,None,NaN,None,16101,None,None,None,Prestwick Capital,None,2.0
11760,None,None,None,None,NaN,None,14381,None,None,None,Lee and Associates,None,2.0
12416,None,None,None,None,NaN,None,15182,None,None,None,Bristlecone Land Use Consulting LLC,Bristlecone Land Use Consulting LLC,2.0
12705,None,None,None,None,NaN,None,15526,Firm,None,None,TBC Land Holding LLC,TBC Land Holding LLC,2.0
11889,None,None,None,None,NaN,None,14552,Firm,None,None,Heavy Metal Construction LLC,Heavy Metal Construction LLC,2.0
11647,None,None,None,None,NaN,None,14233,Firm,None,None,Herringbone Custom Homes LLC,Herringbone Custom Homes LLC,2.0


In [158]:
set(full_table['entity_category'])

{'Agency',
 'Appraisal',
 'Architect',
 'Builder',
 'Contractor',
 'Engineering',
 'Homeowners Association',
 'Inspection',
 None,
 'Other',
 'Real Estate',
 'Surveying',
 'Title'}

In [159]:
set(full_table['entity_subgroup_id'])

{'Ancillary',
 'Asphalt',
 'Asphalt, Excavation',
 'Assessor',
 'Audio Visual',
 'Awnings',
 'Blasting',
 'Boring',
 'Cabinets',
 'Carpentry',
 'Civil',
 'Civil, Planner',
 'Civil, Planner, Landscape',
 'Civil, Planner, Surveying',
 'Civil, Planning',
 'Civil, Planning, Management',
 'Civil, Planning, Surveying',
 'Civil, Structural',
 'Civil, Surveying',
 'Civil, Surveying, Land Planning, Architect',
 'Civil, Surveying, Planning, Landscape',
 'Cleaning',
 'Coatings',
 'Commercial',
 'Communications',
 'Concrete',
 'Concrete, General',
 'Consulting',
 'Controls',
 'Decks, Patio Covers',
 'Demolition',
 'Design',
 'Drywall',
 'Drywall, Remodeling',
 'Economic Development',
 'Electrical',
 'Electrical, Controls',
 'Electrical, General',
 'Electrical, Lighting',
 'Electrical, Mechanical',
 'Electrical, Remodeling',
 'Electrical, Solar',
 'Elevator',
 'Energy',
 'Energy Management',
 'Environmental',
 'Environmental, Civil, Water',
 'Environmental, Water',
 'Erosion Control',
 'Excavating'